## Joining Data Sets

Let us perform few tasks to understand how to perform joins over multiple collections using loops and conditionals.
* There are different strategies for joins.
  * Nested Loops
  * Sort Merge
  * Hash Join
* We will be using Nested Loops approach using orders and order_items.
  * Build dict for one data set - orders.
  * Iteratively lookup into the orders data set while processing the other one - order_items
* Develop a function get_daily_revenue which takes orders, order_items and order_status as arguments and return dict containing order_date and order_revenue. We need to get revenue considering only those orders which satisfy the status passed.

In [28]:
%run 07_preparing_data_sets.ipynb

In [7]:
orders[:10]

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE',
 '6,2013-07-25 00:00:00.0,7130,COMPLETE',
 '7,2013-07-25 00:00:00.0,4530,COMPLETE',
 '8,2013-07-25 00:00:00.0,2911,PROCESSING',
 '9,2013-07-25 00:00:00.0,5657,PENDING_PAYMENT',
 '10,2013-07-25 00:00:00.0,5648,PENDING_PAYMENT']

In [8]:
len(orders)

68883

In [9]:
order_items[:10]

['1,1,957,1,299.98,299.98',
 '2,2,1073,1,199.99,199.99',
 '3,2,502,5,250.0,50.0',
 '4,2,403,1,129.99,129.99',
 '5,4,897,2,49.98,24.99',
 '6,4,365,5,299.95,59.99',
 '7,4,502,3,150.0,50.0',
 '8,4,1014,4,199.92,49.98',
 '9,5,957,1,299.98,299.98',
 '10,5,365,5,299.95,59.99']

In [10]:
len(order_items)

172198

In [11]:
def get_orders_dict(orders, order_status):
    orders_dict = {}
    for order in orders:
        order_details = order.split(',')
        if order_details[3] == order_status:
            orders_dict[int(order_details[0])] = order_details[1]
    return orders_dict

In [12]:
list(get_orders_dict(orders, 'COMPLETE').items())[:10]

[(3, '2013-07-25 00:00:00.0'),
 (5, '2013-07-25 00:00:00.0'),
 (6, '2013-07-25 00:00:00.0'),
 (7, '2013-07-25 00:00:00.0'),
 (15, '2013-07-25 00:00:00.0'),
 (17, '2013-07-25 00:00:00.0'),
 (22, '2013-07-25 00:00:00.0'),
 (26, '2013-07-25 00:00:00.0'),
 (28, '2013-07-25 00:00:00.0'),
 (32, '2013-07-25 00:00:00.0')]

In [14]:
get_orders_dict(orders, 'COMPLETE')[3]

dict

In [10]:
get_orders_dict(orders, 'COMPLETE').get(3)

'2013-07-25 00:00:00.0'

In [15]:
len(get_orders_dict(orders, 'COMPLETE'))

22899

In [ ]:
orders_dict = get_orders_dict(orders, 'COMPLETE')
orders_dict[17]

In [13]:
order_item = order_items[0]
order_item

'1,1,957,1,299.98,299.98'

In [14]:
int(order_item.split(',')[1]) # order_id

1

In [15]:
int(order_item.split(',')[1]) in orders_dict

False

In [16]:
3 in orders_dict

True

In [18]:
def get_daily_revenue(orders, order_items, order_status):
    orders_dict = get_orders_dict(orders, order_status) # dict with order_id as key and order_date as value
    daily_revenue = {} # Initializing empty dict to have date and corresponding revenue
    for order_item in order_items: 
        # Processing all order_items
        order_item_order_id = int(order_item.split(',')[1])
        order_item_subtotal = float(order_item.split(',')[4])
        
        if order_item_order_id in orders_dict:
            # Look up into orders_dict using order_id to get order_date
            orders_dict_date = orders_dict[order_item_order_id] 
            if orders_dict_date in daily_revenue:
                # If order_date exists in orders_dict then add order_item_subtotal to the existing value
                # based up on the key in daily_revenue
                daily_revenue[orders_dict_date] = round(daily_revenue[orders_dict_date] + order_item_subtotal, 2)
            else:
                # If order_date does not exists then add new element to daily_revenue
                daily_revenue[orders_dict_date] = order_item_subtotal
    return daily_revenue

In [19]:
orders_sample = [
    '4,2013-07-25 00:00:00.0,8827,CLOSED',
    '5,2013-07-25 00:00:00.0,11318,COMPLETE',
    '7,2013-07-25 00:00:00.0,4530,COMPLETE',
    '105,2013-07-26 00:00:00.0,8220,COMPLETE'
]

In [20]:
order_items_sample = [
    '5,4,897,2,49.98,24.99',
    '6,4,365,5,299.95,59.99',
    '7,4,502,3,150.0,50.0',
    '8,4,1014,4,199.92,49.98',
    '9,5,957,1,299.98,299.98',
    '10,5,365,5,299.95,59.99',
    '11,5,1014,2,99.96,49.98',
    '12,5,957,1,299.98,299.98',
    '13,5,403,1,129.99,129.99',
    '14,7,1073,1,199.99,199.99',
    '15,7,957,1,299.98,299.98',
    '16,7,926,5,79.95,15.99',
    '239,105,403,1,129.99,129.99',
    '240,105,365,3,179.97,59.99',
    '241,105,365,2,119.98,59.99'
]

In [21]:
get_daily_revenue(orders_sample, order_items_sample, 'COMPLETE')

{'2013-07-25 00:00:00.0': 1709.78, '2013-07-26 00:00:00.0': 429.94}

```{note}
Use the function to get daily revenue considering only COMPLETE orders.
```

In [22]:
get_daily_revenue(orders, order_items, 'COMPLETE')

{'2013-07-25 00:00:00.0': 20030.32,
 '2013-07-26 00:00:00.0': 42165.88,
 '2013-07-27 00:00:00.0': 33156.21,
 '2013-07-28 00:00:00.0': 27012.91,
 '2013-07-29 00:00:00.0': 45898.65,
 '2013-07-30 00:00:00.0': 40590.21,
 '2013-07-31 00:00:00.0': 46503.83,
 '2013-08-01 00:00:00.0': 38231.41,
 '2013-08-02 00:00:00.0': 36633.44,
 '2013-08-03 00:00:00.0': 34828.71,
 '2013-08-04 00:00:00.0': 26161.97,
 '2013-08-05 00:00:00.0': 25804.33,
 '2013-08-06 00:00:00.0': 41413.79,
 '2013-08-07 00:00:00.0': 31533.1,
 '2013-08-08 00:00:00.0': 27359.07,
 '2013-08-09 00:00:00.0': 22091.95,
 '2013-08-10 00:00:00.0': 39038.72,
 '2013-08-11 00:00:00.0': 21302.69,
 '2013-08-12 00:00:00.0': 41139.11,
 '2013-08-13 00:00:00.0': 12468.53,
 '2013-08-14 00:00:00.0': 29760.88,
 '2013-08-15 00:00:00.0': 37473.12,
 '2013-08-16 00:00:00.0': 25547.74,
 '2013-08-17 00:00:00.0': 52940.94,
 '2013-08-18 00:00:00.0': 31304.59,
 '2013-08-19 00:00:00.0': 10603.87,
 '2013-08-20 00:00:00.0': 32680.05,
 '2013-08-21 00:00:00.0': 203

In [23]:
len(get_daily_revenue(orders, order_items, 'COMPLETE'))

364

In [24]:
get_daily_revenue(orders, order_items, 'COMPLETE')['2014-07-10 00:00:00.0']

30247.47